# Introduction to metaprogramming

Normal programs manipulate data. Metaprograms **manipulate code**. (In many languages this is impossible.)
Of course, the ultimate goal is that the programs that we manipulate will later operate on data.

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "programming a program"

i.e. writing code (a program) to manipulate not data, but code. (And this code will, itself, then manipulate data [or another program...].)

Conjecture: metaprogramming, not taught much in scientific computing, is what we need more of as optimization, autodiff, and dimension reduction is not operating on whole programs.

## Symbols and Expressions

In [ ]:
:a, :b, :c, :foo, :α, :call # These are all symbols

In [ ]:
:(a+1),:(sin(b)),:(c(10)),:(foo(a,b,c)),:(foo(α^2+1)) # These are all expressions

A really good explanation of the what and why of symbols in Julia: [Stack Overflow](https://stackoverflow.com/questions/23480722/what-is-a-symbol-in-julia)

We use the special syntax `:( ... )` to represent a piece of Julia code.  It is called quoting.

In [ ]:
ex = :(x+3)

We can see that the object `ex` is composied of objects and symbols:

In [ ]:
dump(ex)

In [ ]:
typeof(ex)

We see that `ex` is of type `Expr`, meaning a Julia expression object - a piece of Julia code, stored as an object inside Julia itself!

The details about the object are stored (as usual) in its fields. The kind of expression object is stored as the `head` field:

In [ ]:
ex.head

In this case, we see that it represents a function call. 

The arguments of the function call are stored in the `ex.args` field:

In [ ]:
ex.args

In [ ]:
ex.args[2]

We can now *manipulate* and *modify* the object by *changing* these fields:

In [ ]:
ex.args[2] = :z
ex.args[3] = :4

In [ ]:
ex

We see that the expression has changed. We can even change it from an addition to something else:

In [ ]:
ex.args[1] = :*

In [ ]:
ex

This is the basis of metaprogramming, since it gives us the ability to take one piece of code, and modify it to produce another piece of code.

The foundation of an expression are `Symbol`s, written with initial colons (`:`):


In [ ]:
typeof(:z)

A more compact form to display an expression is as follows:

In [ ]:
using Pkg

In [ ]:
Pkg.add("MacroTools")

In [ ]:
using MacroTools

In [ ]:
Meta.show_sexpr(ex)

## More complicated expressions

What about more complicated expressions?

In [ ]:
ex2 = :(x + 3y)

In [ ]:
Meta.show_sexpr(ex2)

In [ ]:
dump(ex2)

In [ ]:
ex2.args[3]

In [ ]:
typeof(ex2.args[3])

We see that expressions are recursive, in the sense that the subpiece `:(3y)` is itself another `Expr` object.

Julia provides a mechanism to create an `Expr` from a `String`, via the `parse` function:

In [ ]:
Meta.parse("x + 3y")

In [ ]:
dump(:(x+3x^2))

# Build Your Own

## Manipulating expressions

How can we do something more complicated? For example, suppose we start with the expression

In [ ]:
ex = :(x + 3x^2)

In [ ]:
dump(ex)

and we would like to change *all* `x`s to `(x+1)`s? We will need to dig down into the structure.
A first attempt is the following function. Note that this *modifies* its argument, so has a `!` ("bang") in the function name.

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        
        if args[i] == :x
            args[i] = :(x+1)
        end
        
    end
end

In [ ]:
traverse!(ex)

In [ ]:
ex

Oops, that didn't work -- we didn't manage to reach inside the inner `Expr` object. 

In [ ]:
ex = :(x + 3x^2)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        if args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [ ]:
traverse!(ex)

The AST ("Abstract Syntax Tree") has a **recursive** structure, so we use a recursive function:

In [ ]:
Meta.show_sexpr(ex)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        
        if isa(args[i], Expr)
            traverse!(args[i])
            
        elseif args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [ ]:
traverse(ex) = ex  # generally do nothing
traverse(ex::Symbol) = ex==:x ? :(x+1) : ex # for the symbol :x make it :(x+1)
traverse(ex::Expr) = Expr(:call,traverse.(ex.args)...) # Expressions are done recursively

In [6]:
traverse!(ex) = ex  # generally do nothing
traverse!(ex::Symbol) = ex==:x ? :(x+1) : ex # for the symbol :x make it :(x+1)
traverse!(ex::Expr) = (ex.args.=traverse!.(ex.args); ex)

traverse! (generic function with 3 methods)

In [5]:
ex = :(x^2+1)
traverse!(ex)
ex

:(nothing + 1)

In [5]:
ex = :(x+y)
foo!(e::Expr) = (ex.args=[:*,:p,:q];nothing)
foo!(e::Symbol) = e==:x ? e=:(x+1) : e
foo!(ex)
ex = :a
foo!(ex)
ex

:a

In [7]:
ex = :(x + 3x^2)
traverse!(ex)

:((x + 1) + 3 * (x + 1) ^ 2)

**Exercise**: Make the `traverse!` function more general, to replace `x` by an arbitrary expression that is another argument of the function.

## Evaluation

Now that we have our new object, we want to use it in Julia. We do this using `eval`:


In [ ]:
eval(x) = Core.eval(Main, x)

In [ ]:
ex1 = :(3x^2)
x=10
eval(ex1)

In [ ]:
op = :^
ex2 = :(3*$op(x,2))
eval(ex2)

In [ ]:
dump(ex1)

In [ ]:
dump(ex2)

In [ ]:
x = 3

In [ ]:
eval(ex)

In [ ]:
op = :^
ex = :(3*$op(x,2))
dump(ex)

## Using multiple dispatch with expressions

What if, say, we want to wrap "number literals" like 3, 4.5 with `f(3)` etc.?
We will write a more Julian version, using **multiple dispatch**:

In [ ]:
function wrap_literals(ex::Expr, f::Symbol)
    
    #println("Entering !")
    #@show ex, s, new_expr
    
    args = ex.args
    
    for i in 1:length(args)
        #@show i, args[i]
        args[i] = wrap_literals(args[i], f)
        
    end
    
    return ex
end

wrap_literals(x::Number, f::Symbol) = :($f($x))
wrap_literals(x, f) = x  # fall-back method

In [ ]:
ex = :(3x^2)
wrap_literals(:(3x^2), :f)

## Inserting pieces of code

Wilkinson-type polynomials are polynomials like

$p_5(x) = (x-1) (x-2) (x-3) (x-4) (x-5)$

In Julia we could write

In [ ]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

**Exercise:** We can interpolate pieces of code in other pieces using the `$` operator, and symbols from strings using `Symbol`. Make and evaluate an expression to define $p_n$ for different $n$, and interact with it.

## Generating repetitive code

One of the uses of metaprogramming is **code generation**. 
Suppose that for some reason we want to make a type that behaves like a `Float64`, but with some additional functionality.:

In [ ]:
struct MyFloat
    a::Float64
end

We would need to write methods like


In [ ]:
import Base: +, -

+(x::MyFloat, y::MyFloat) = x.a + y.a
-(x::MyFloat, y::MyFloat) = x.a - y.a

This will get boring fast. Whenever we have repetition like this, we should get the computer to do it for us. However, here we need to **repeat code**. Let's make a code *template*; it should look like

    op(x::MyFloat, y::MyFloat) = op(x.a, y.a)

We want a code object that looks like that:

In [ ]:
ex =(:(op(x::MyFloat, y::MyFloat) = op(x.a, y.a)))

We can also write this as 

In [ ]:
ex = quote
        op(x::MyFloat, y::MyFloat) = op(x.a, y.a)
    end 

However, we want to replace `op` **by its value**

In [ ]:
op = :+
ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end 

And now just loop:

In [ ]:
import Base: +,-,*,/

for op in (:+, :-, :*, :/)
    
    ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end
    
    eval(ex)
end


Julia provides `@eval` to make this shorter (by not needing the quote):

In [ ]:
for op in (:+, :-, :*, :/)
    
    @eval $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    
end


In [ ]:
 x=1
 y=1
 op = :+
 @eval  a=$op(x, y) 
 a

In [ ]:
@eval x=2

In [ ]:
eval(:(x=2))

In [ ]:
op = :^
ex = :(3*$op(x,2))
eval(ex)
eval(:(3*$op(x,2)))

## Macros

We don't want the user to have to think about Julia syntax objects. We can supply a **macro**.
This can be thought of as a "meta-function": it takes a Julia expression object as argument, and returns a Julia expression object. However, when it is called, it *automatically parses its input into the expression object*.

The simplest example:

In [ ]:
macro simple(ex)
    println(ex)
    println(typeof(ex))
    show(dump(ex))
    return ex
end
    

In [ ]:
@simple a=[1,2,3]

We can now make a macro `wrap` that calls our function:

In [ ]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    show(new_ex)
end

In [ ]:
@wrap g 3x+2

We can also call it as `@wrap(g, 3x+2)`:

In [ ]:
@wrap(g, 3x+2)

In fact, macros should **return an expression**. This expression will be **evaluated in the scope in which it is called**:

In [ ]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    return new_ex
end

In [ ]:
@wrap(g, 3x+2)

In [ ]:
x = 3
g(x) = 10x

In [ ]:
@wrap g 3x+2

In [ ]:
g(3)*x+g(2)

In [ ]:
macro playwith(e1,e2)
    show(dump(e1))
    show(dump(e2))
    return(e1)
end

In [ ]:
@playwith 1+1 2+2

In [ ]:
a = rand(3)'
dump(a)

In [ ]:
supertype(typeof(a))

In [ ]:
a = rand(3)'
? sum

In [ ]:
?sum

In [ ]:
sum(a,2)

In [ ]:
a = rand(5)

In [ ]:
a = rand(2,2)
?norm

In [ ]:
 ?opnorm